<a href="https://colab.research.google.com/github/badtripdude/NN_FineTuning_FlowerDataset102/blob/main/NN_FineTuning_FlowerDataset102.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
!pip install -q kaggle

In [41]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"ratmirshagivaleev","key":"55618b233f48e924c1a5be9175f3a5fb"}'}

In [42]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [43]:
!chmod 600 /root/.kaggle/kaggle.json

In [44]:
!ls
# https://www.kaggle.com/datasets/demonplus/flower-dataset-102/download/ivxdtU9A8KqyvqecHcdM%2Fversions%2FP2Q2VkAR9a5LnovH0OVf%2Ffiles%2F102flowers.tgz?datasetVersionNumber=1

 102flowers.tgz    flower-dataset-102.zip  'kaggle (1).json'
 best_model.hdf5   imagelabels.mat	    kaggle.json
 data		   jpg			    sample_data


In [45]:
!kaggle datasets list

ref                                                             title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
victorsoeiro/netflix-tv-shows-and-movies                        Netflix TV Shows and Movies                           2MB  2022-05-15 00:01:23          10070        307  1.0              
devansodariya/student-performance-data                          Student Performance Dataset                           7KB  2022-05-26 13:55:09           5542        172  0.9705882        
mohamedharris/supermart-grocery-sales-retail-analytics-dataset  Supermart Grocery Sales - Retail Analytics Dataset  191KB  2022-06-12 16:14:44            812         44  0.88235295       
sameepvani/nasa-nearest-earth-objects                       

In [46]:
!kaggle datasets download demonplus/flower-dataset-102

flower-dataset-102.zip: Skipping, found more recently modified local copy (use --force to force download)


In [47]:
!ls

 102flowers.tgz    flower-dataset-102.zip  'kaggle (1).json'
 best_model.hdf5   imagelabels.mat	    kaggle.json
 data		   jpg			    sample_data


In [48]:
!unzip flower-dataset-102.zip
!mkdir data

!cp 102flowers.tgz *.mat data
!ls data

Archive:  flower-dataset-102.zip
replace 102flowers.tgz? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: 102flowers.tgz          
replace imagelabels.mat? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: imagelabels.mat         
mkdir: cannot create directory ‘data’: File exists
102flowers.tgz	imagelabels.mat


In [49]:
!ls data

102flowers.tgz	imagelabels.mat


In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import scipy.io
import tarfile
import csv
import sys
import os


import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.models as M
import tensorflow.keras.layers as L
import tensorflow.keras.backend as K
import tensorflow.keras.callbacks as C
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, EarlyStopping
from tensorflow.keras.callbacks import Callback
#from tensorflow.keras.regularizers import l2
from tensorflow.keras import optimizers
#from tf.keras.applications.inception_v3 import Inception_v3

from sklearn.model_selection import train_test_split
 
import PIL
from PIL import ImageOps, ImageFilter

In [51]:
EPOCHS               = 12
BATCH_SIZE           = 128
LR                   = 1e-3
VAL_SPLIT            = 0.2

CLASS_NUM            = 102
IMG_SIZE             = 128
IMG_CHANNELS         = 3
input_shape          = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)

DATA_PATH = 'data/'
PATH = "./"

In [52]:
# Setting seed for reproducibility
# os.makedirs(PATH,exist_ok=False)

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)  
PYTHONHASHSEED = 0

In [53]:
# прочитаем все названия файлов прям из архива и таргеты к ним
def get_all_filenames(tar_fn):
    with tarfile.open(tar_fn) as f:
        return [m.name for m in f.getmembers() if m.isfile()]

df = pd.DataFrame()
df['Id'] = sorted(get_all_filenames(DATA_PATH+"102flowers.tgz"))
df['Category'] = scipy.io.loadmat(DATA_PATH+'imagelabels.mat')['labels'][0] - 1  # классы (0, 1, 2, ...)
df['Category'] = df['Category'].astype(str)

In [54]:
tarfile.open(DATA_PATH + '102flowers.tgz').extractall()


In [55]:
!ls 

 102flowers.tgz    flower-dataset-102.zip  'kaggle (1).json'
 best_model.hdf5   imagelabels.mat	    kaggle.json
 data		   jpg			    sample_data


In [56]:
df.head(5)

,Id,Category
0,jpg/image_00001.jpg,76
1,jpg/image_00002.jpg,76
2,jpg/image_00003.jpg,76
3,jpg/image_00004.jpg,76
4,jpg/image_00005.jpg,76


In [57]:
train_files, test_files, train_labels, test_labels = \
    train_test_split(df['Id'], df['Category'], test_size=0.2, random_state=42, stratify=df['Category'])

train_files = pd.DataFrame(train_files)
test_files = pd.DataFrame(test_files)
train_files['Category'] = train_labels
test_files['Category'] = test_labels

train_files.shape, test_files.shape

((6551, 2), (1638, 2))

In [58]:
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   rotation_range = 50,
                                   shear_range=0.2,
                                   zoom_range=[0.75, 1.25],
                                   brightness_range=[0.5, 1.5],
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   horizontal_flip=True
                                  )

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [59]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_files,
    directory=PATH,
    x_col="Id",
    y_col="Category",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, 
    seed=RANDOM_SEED,)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_files,
    directory=PATH,
    x_col="Id",
    y_col="Category",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False, 
    seed=RANDOM_SEED,)

Found 6551 validated image filenames belonging to 102 classes.
Found 1638 validated image filenames belonging to 102 classes.


In [60]:
model_base = keras.applications.ResNet50V2(weights='imagenet', include_top=False, input_shape=input_shape)

In [61]:
model_base.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 134, 134, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 64, 64, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 pool1_pad (ZeroPadding2D)      (None, 66, 66, 64)   0           ['conv1_conv[0][0]']    

In [62]:
# first: train only top layers (which were randomly initialized) step 1!
model_base.trainable = False

In [63]:
model = M.Sequential()
model.add(model_base)
model.add(L.GlobalAveragePooling2D(),)
model.add(L.Dense(102, activation='softmax'))

In [64]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 4, 4, 2048)        23564800  
                                                                 
 global_average_pooling2d_1   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_1 (Dense)             (None, 102)               208998    
                                                                 
Total params: 23,773,798
Trainable params: 208,998
Non-trainable params: 23,564,800
_________________________________________________________________


In [65]:
len(model.layers)

3

In [66]:
len(model.trainable_variables)

2

In [67]:
for layer in model.layers:
  print(layer, layer.trainable)

<keras.engine.functional.Functional object at 0x7fd5f0459e50> False
<keras.layers.pooling.GlobalAveragePooling2D object at 0x7fd5f03dad90> True
<keras.layers.core.dense.Dense object at 0x7fd5f05319d0> True


# Fit


# step 1

In [68]:
LR = 0.001
model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adamax(learning_rate=LR), metrics=["accuracy"])

In [69]:
checkpoint = ModelCheckpoint('best_model.hdf5' , monitor = ['val_accuracy'] , verbose = 1  , mode = 'max')
earlystop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
callbacks_list = [checkpoint, earlystop]

In [70]:
scores = model.evaluate(test_generator, steps=1, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

1/1 [==============================] - 2s 2s/step - loss: 5.9450 - accuracy: 0.0234
Accuracy: 2.34%


In [71]:
history = model.fit(
        train_generator,
        steps_per_epoch = train_generator.samples//train_generator.batch_size,
        validation_data = test_generator, 
        validation_steps = test_generator.samples//test_generator.batch_size,
        epochs = EPOCHS,
        callbacks = callbacks_list,
)

Epoch 1/12
51/51 [==============================] - ETA: 0s - loss: 3.8449 - accuracy: 0.1618
Epoch 1: saving model to best_model.hdf5
51/51 [==============================] - 121s 1s/step - loss: 3.8449 - accuracy: 0.1618 - val_loss: 2.7260 - val_accuracy: 0.3503
Epoch 2/12
51/51 [==============================] - ETA: 0s - loss: 2.4981 - accuracy: 0.3913
Epoch 2: saving model to best_model.hdf5
51/51 [==============================] - 62s 1s/step - loss: 2.4981 - accuracy: 0.3913 - val_loss: 2.0908 - val_accuracy: 0.5072
Epoch 3/12
51/51 [==============================] - ETA: 0s - loss: 2.0366 - accuracy: 0.4985
Epoch 3: saving model to best_model.hdf5
51/51 [==============================] - 63s 1s/step - loss: 2.0366 - accuracy: 0.4985 - val_loss: 1.7895 - val_accuracy: 0.5872
Epoch 4/12
51/51 [==============================] - ETA: 0s - loss: 1.7998 - accuracy: 0.5597
Epoch 4: saving model to best_model.hdf5
51/51 [==============================] - 62s 1s/step - loss: 1.7998 - ac

In [72]:
model.save('data/model_last.hdf5')
model.load_weights('best_model.hdf5')

In [73]:
scores = model.evaluate_generator(test_generator, verbose=1)
print('accuracy: %.2f%%' % (scores[1] * 100))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


13/13 [==============================] - 9s 710ms/step - loss: 1.0885 - accuracy: 0.7277
accuracy: 72.77%


# step 2

In [74]:
model_base.trainable = True
fine_tune_at = 100

for layer in model_base.layers[:fine_tune_at]:
  layer.trainable = False

In [75]:
len(model_base.trainable_variables)

82

In [76]:
LR = 0.0001
model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adamax(learning_rate=LR), metrics=["accuracy"])

In [77]:
checkpoint = ModelCheckpoint('best_model.hdf5' , monitor = ['val_accuracy'] , verbose = 1  , mode = 'max')
earlystop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
callbacks_list = [checkpoint, earlystop]

In [78]:
scores = model.evaluate(test_generator, steps=1, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

1/1 [==============================] - 2s 2s/step - loss: 1.0607 - accuracy: 0.7266
Accuracy: 72.66%


In [79]:
history = model.fit(
        train_generator,
        steps_per_epoch = train_generator.samples//train_generator.batch_size,
        validation_data = test_generator, 
        validation_steps = test_generator.samples//test_generator.batch_size,
        epochs = 10,
        callbacks = callbacks_list,
)

Epoch 1/10
51/51 [==============================] - ETA: 0s - loss: 2.0566 - accuracy: 0.5200
Epoch 1: saving model to best_model.hdf5
51/51 [==============================] - 69s 1s/step - loss: 2.0566 - accuracy: 0.5200 - val_loss: 1.5409 - val_accuracy: 0.5833
Epoch 2/10
51/51 [==============================] - ETA: 0s - loss: 1.2580 - accuracy: 0.6886
Epoch 2: saving model to best_model.hdf5
51/51 [==============================] - 65s 1s/step - loss: 1.2580 - accuracy: 0.6886 - val_loss: 1.1598 - val_accuracy: 0.6784
Epoch 3/10
51/51 [==============================] - ETA: 0s - loss: 0.9583 - accuracy: 0.7619
Epoch 3: saving model to best_model.hdf5
51/51 [==============================] - 65s 1s/step - loss: 0.9583 - accuracy: 0.7619 - val_loss: 0.8873 - val_accuracy: 0.7572
Epoch 4/10
51/51 [==============================] - ETA: 0s - loss: 0.8078 - accuracy: 0.7973
Epoch 4: saving model to best_model.hdf5
51/51 [==============================] - 65s 1s/step - loss: 0.8078 - acc

In [80]:
model.save('data/model_last.hdf5')
model.load_weights('best_model.hdf5')

In [81]:
scores = model.evaluate_generator(test_generator, verbose=1)
print('accuracy: %.2f%%' % (scores[1] * 100))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


13/13 [==============================] - 8s 593ms/step - loss: 0.5109 - accuracy: 0.8632
accuracy: 86.32%


# step 3

In [82]:
model_base.trainable = True
fine_tune_at = 50
for layer in model_base.layers[:fine_tune_at]:
  layer.trainable = False
len(model_base.trainable_variables)

128

In [83]:
LR = 0.00001
model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adamax(learning_rate=LR), metrics=["accuracy"])

In [84]:
checkpoint = ModelCheckpoint('best_model.hdf5' , monitor = ['val_accuracy'] , verbose = 1  , mode = 'max')
earlystop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
callbacks_list = [checkpoint, earlystop]

In [85]:
scores = model.evaluate(test_generator, steps=1, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

1/1 [==============================] - 2s 2s/step - loss: 0.4426 - accuracy: 0.8750
Accuracy: 87.50%


In [86]:
history = model.fit(
        train_generator,
        steps_per_epoch = train_generator.samples//train_generator.batch_size,
        validation_data = test_generator, 
        validation_steps = test_generator.samples//test_generator.batch_size,
        epochs = 10,
        callbacks = callbacks_list,
)

Epoch 1/10
51/51 [==============================] - ETA: 0s - loss: 0.4304 - accuracy: 0.8912
Epoch 1: saving model to best_model.hdf5
51/51 [==============================] - 71s 1s/step - loss: 0.4304 - accuracy: 0.8912 - val_loss: 0.5092 - val_accuracy: 0.8607
Epoch 2/10
51/51 [==============================] - ETA: 0s - loss: 0.4102 - accuracy: 0.8946
Epoch 2: saving model to best_model.hdf5
51/51 [==============================] - 66s 1s/step - loss: 0.4102 - accuracy: 0.8946 - val_loss: 0.5144 - val_accuracy: 0.8594
Epoch 3/10
51/51 [==============================] - ETA: 0s - loss: 0.3990 - accuracy: 0.8994
Epoch 3: saving model to best_model.hdf5
51/51 [==============================] - 66s 1s/step - loss: 0.3990 - accuracy: 0.8994 - val_loss: 0.5175 - val_accuracy: 0.8600
Epoch 4/10
51/51 [==============================] - ETA: 0s - loss: 0.3866 - accuracy: 0.9044
Epoch 4: saving model to best_model.hdf5
51/51 [==============================] - 67s 1s/step - loss: 0.3866 - acc

In [87]:
model.save('data/model_last.hdf5')
model.load_weights('best_model.hdf5')

In [88]:
scores = model.evaluate_generator(test_generator, verbose=1)
print('accuracy: %.2f%%' % (scores[1] * 100))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


13/13 [==============================] - 8s 588ms/step - loss: 0.5049 - accuracy: 0.8651
accuracy: 86.51%


# step 4

In [89]:
model_base.trainable = True
LR = 0.00001
model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adamax(learning_rate=LR), metrics=["accuracy"])

In [90]:
checkpoint = ModelCheckpoint('best_model.hdf5' , monitor = ['val_accuracy'] , verbose = 1  , mode = 'max')
earlystop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
callbacks_list = [checkpoint, earlystop]

In [91]:
scores = model.evaluate(test_generator, steps=1, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

1/1 [==============================] - 2s 2s/step - loss: 0.4036 - accuracy: 0.8906
Accuracy: 89.06%


In [92]:
history = model.fit(
        train_generator,
        steps_per_epoch = train_generator.samples//train_generator.batch_size,
        validation_data = test_generator, 
        validation_steps = test_generator.samples//test_generator.batch_size,
        epochs = 15,
        callbacks = callbacks_list,
)

Epoch 1/15
51/51 [==============================] - ETA: 0s - loss: 0.3755 - accuracy: 0.9027
Epoch 1: saving model to best_model.hdf5
51/51 [==============================] - 75s 1s/step - loss: 0.3755 - accuracy: 0.9027 - val_loss: 0.5372 - val_accuracy: 0.8535
Epoch 2/15
51/51 [==============================] - ETA: 0s - loss: 0.3469 - accuracy: 0.9141
Epoch 2: saving model to best_model.hdf5
51/51 [==============================] - 69s 1s/step - loss: 0.3469 - accuracy: 0.9141 - val_loss: 0.5372 - val_accuracy: 0.8490
Epoch 3/15
51/51 [==============================] - ETA: 0s - loss: 0.3388 - accuracy: 0.9120
Epoch 3: saving model to best_model.hdf5
51/51 [==============================] - 69s 1s/step - loss: 0.3388 - accuracy: 0.9120 - val_loss: 0.5206 - val_accuracy: 0.8496
Epoch 4/15
51/51 [==============================] - ETA: 0s - loss: 0.3257 - accuracy: 0.9197
Epoch 4: saving model to best_model.hdf5
51/51 [==============================] - 69s 1s/step - loss: 0.3257 - acc

In [93]:
model.save('data/model_last.hdf5')
model.load_weights('best_model.hdf5')

In [94]:
scores = model.evaluate_generator(test_generator, verbose=1)
print('accuracy: %.2f%%' % (scores[1] * 100))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


13/13 [==============================] - 8s 589ms/step - loss: 0.4744 - accuracy: 0.8657
accuracy: 86.57%


# step 5

In [95]:
IMG_SIZE = 320
IMG_CHANNELS = 3

LR = 1e-4
BATCH_SIZE = 64

In [98]:
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                  #  rotation_range = 50,
                                  #  shear_range=0.2,
                                   zoom_range=[0.75, 1.25],
                                  #  brightness_range=[0.5, 1.5],
                                  #  width_shift_range=0.1,
                                  #  height_shift_range=0.1,
                                   horizontal_flip=True
                                  )

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [99]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_files,
    directory=PATH,
    x_col="Id",
    y_col="Category",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, 
    seed=RANDOM_SEED,)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_files,
    directory=PATH,
    x_col="Id",
    y_col="Category",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False, 
    seed=RANDOM_SEED,)

Found 6551 validated image filenames belonging to 102 classes.
Found 1638 validated image filenames belonging to 102 classes.


In [116]:
model_base = keras.applications.ResNet50V2(weights='imagenet', include_top=False, input_shape=input_shape)

In [118]:
model_base.trainable = True

In [119]:
model = M.Sequential()
model.add(model_base)
model.add(L.GlobalAveragePooling2D(),)
model.add(L.Dense(102, activation='softmax'))

In [120]:
model.load_weights('best_model.hdf5')

In [121]:
model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adamax(learning_rate=LR), metrics=["accuracy"])

In [122]:
scores = model.evaluate_generator(test_generator, verbose=1)
print('accuracy: %.2f%%' % (scores[1] * 100))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


26/26 [==============================] - 14s 464ms/step - loss: 2.8967 - accuracy: 0.5433
accuracy: 54.33%


In [123]:
history = model.fit(
        train_generator,
        steps_per_epoch = train_generator.samples//train_generator.batch_size,
        validation_data = test_generator, 
        validation_steps = test_generator.samples//test_generator.batch_size,
        epochs = 10,
        callbacks = callbacks_list,
)

Epoch 1/10
102/102 [==============================] - ETA: 0s - loss: 0.3985 - accuracy: 0.9243
Epoch 1: saving model to best_model.hdf5
102/102 [==============================] - 222s 2s/step - loss: 0.3985 - accuracy: 0.9243 - val_loss: 0.2986 - val_accuracy: 0.9294
Epoch 2/10
102/102 [==============================] - ETA: 0s - loss: 0.1239 - accuracy: 0.9824
Epoch 2: saving model to best_model.hdf5
102/102 [==============================] - 209s 2s/step - loss: 0.1239 - accuracy: 0.9824 - val_loss: 0.1770 - val_accuracy: 0.9575
Epoch 3/10
102/102 [==============================] - ETA: 0s - loss: 0.0711 - accuracy: 0.9928
Epoch 3: saving model to best_model.hdf5
102/102 [==============================] - 211s 2s/step - loss: 0.0711 - accuracy: 0.9928 - val_loss: 0.1532 - val_accuracy: 0.9619
Epoch 4/10
102/102 [==============================] - ETA: 0s - loss: 0.0470 - accuracy: 0.9960
Epoch 4: saving model to best_model.hdf5
102/102 [==============================] - 210s 2s/step 

In [124]:
model.save('data/model_last.hdf5')
model.load_weights('best_model.hdf5')

In [125]:
scores = model.evaluate_generator(test_generator, verbose=1)
print('accuracy: %.2f%%' % (scores[1] * 100))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


26/26 [==============================] - 12s 462ms/step - loss: 0.1123 - accuracy: 0.9689
accuracy: 96.89%
